# Fitting a sine curve with a neural network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(0)

# Generate training data
x_data = torch.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_data = torch.sin(x_data)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(x_data, y_data)
batch_size = len(dataset)

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define a simple neural network
class SineNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(1, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
    
    def forward(self, x_input):
        x_processed = F.relu(self.linear1(x_input))
        x_processed = F.relu(self.linear2(x_processed))
        x_processed = self.linear3(x_processed)
        return x_processed

# Initialize model, loss function and optimizer
model = SineNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = 0
    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1
    
    average_epoch_loss = epoch_loss / num_batches
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_epoch_loss:.4f}')

# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(x_data.numpy(), y_data.numpy(), label='True sine')
# Pass the original x_data tensor for a coherent plot of the learned function
plt.scatter(x_data.numpy(), model(x_data).detach().numpy(), label='Predicted sine')
plt.legend()
plt.title('Sine Function Approximation (DataLoader Approach)')
plt.show()

# Adding a learning rate scheduler

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(0)

# Generate training data
x_data = torch.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_data = torch.sin(x_data)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(x_data, y_data)
batch_size = 64 # len(dataset)

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define a simple neural network
class SineNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(1, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

# Initialize model, loss function and optimizer
model = SineNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    epoch_loss = 0.0
    num_batches = 0

    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1

    scheduler.step()
    
    average_epoch_loss = epoch_loss / num_batches
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Avg Loss: {average_epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.5f}')

# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(x_data.numpy(), y_data.numpy(), label='True sine')
# Pass the original x_data tensor for a coherent plot of the learned function
plt.scatter(x_data.numpy(), model(x_data).detach().numpy(), label='Predicted sine')
plt.legend()
plt.title('Sine Function Approximation (DataLoader Approach)')
plt.show()

# Storing predictions every Nth epoch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
# import os # No longer needed for intermediate plot checkpoint directory

torch.manual_seed(0)

N_plot_interval = 500
history_predictions = {}

# Generate training data
x_data = torch.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_data = torch.sin(x_data)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(x_data, y_data)
batch_size = 64
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define a simple neural network
class SineNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(1, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

# Initialize model, loss function and optimizer
model = SineNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

# Training loop
num_epochs = 5000
print("Starting training...")
for epoch in range(num_epochs):
    model.train() # Ensure model is in training mode
    epoch_loss = 0.0
    num_batches = 0

    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1

    scheduler.step()
    
    average_epoch_loss = epoch_loss / num_batches
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Avg Loss: {average_epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.5f}')

    # Store Predictions at Nth epoch
    if (epoch + 1) % N_plot_interval == 0:
        model.eval() # Switch to evaluation mode
        with torch.no_grad(): # Disable gradient calculations
            predictions_at_interval = model(x_data)

        history_predictions[epoch + 1] = predictions_at_interval.detach().cpu().numpy()
        print(f'--- Stored predictions for plotting at Epoch {epoch+1} ---')
        model.train() 

print("Training finished.")

# --- Plotting All Stored Predictions After Training ---
print("\nGenerating historical plots...")

# Ensure the very final state is also available for plotting if not already captured
model.eval()
with torch.no_grad():
    final_predictions_np = model(x_data).detach().cpu().numpy()
# Add/overwrite the last epoch's predictions to ensure it's the absolute final state
history_predictions[num_epochs] = final_predictions_np 

# Sort epochs to plot them in order
sorted_epochs = sorted(history_predictions.keys())

for epoch_num in sorted_epochs:
    predictions_np = history_predictions[epoch_num]
    
    plt.figure(figsize=(10, 6))
    plt.scatter(x_data.numpy(), y_data.numpy(), label='True sine', s=20, alpha=0.7, edgecolors='k', linewidths=0.5)
    plt.scatter(x_data.numpy(), predictions_np, label=f'Predicted (Epoch {epoch_num})', s=20, alpha=0.7)
    plt.legend()
    plt.title(f'Sine Function Approximation - Epoch {epoch_num}')
    plt.xlabel('x')
    plt.ylabel('sin(x)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ylim(-1.1, 1.1)
    plt.show()
print("All historical plots generated.")